# **Telecom Customer Churn Prediction Feature Engineering**

This notebook constructs a feature matrix ready for predictive modeling.  Steps include data profiling, categorical encoding, a rich suite of derived and interaction features, quality-control filtering, multicollinearity checks, dimensionality reduction, and a preliminary feature-importance ranking.


In [1]:
!pip install missingno numpy pandas plotly scikit-learn statsmodels

In [2]:
import os

import missingno as msno
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.api as sm

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Plotly defaults for consistent interactivity
px.defaults.template = "plotly_white"
px.defaults.width = 750
px.defaults.height = 500

##  **Data Ingestion & Initial Inspection**

Dataset retrieval, basic dimensions, and schema verification.


In [3]:
raw_fp = '../data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv'
df_raw = pd.read_csv(raw_fp)


print(f"Dataset shape: {df_raw.shape[0]:,} rows × {df_raw.shape[1]:,} columns")
display(df_raw.head(3))

Dataset shape: 7,043 rows × 21 columns


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


##  **Data Quality Audit**

###  **Data types, missing counts, duplicate count**


In [4]:
info_buf = df_raw.info()
nulls = df_raw.isnull().sum()
dups = df_raw.duplicated().sum()

print("\nMissing values per column:\n", nulls[nulls>0])
print(f"\nTotal duplicates: {dups}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


No duplicate customer records were found; however, the TotalCharges field is stored as an object (string) and must be converted to a numeric type.

## **Change TotalCharges to Numeric**
Coerce TotalCharges to numeric and impute with median to maintain central tendency and avoid outlier distortion.


In [5]:
df = df_raw.copy()
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

null_before = df['TotalCharges'].isnull().sum()
median_tc   = df['TotalCharges'].median()
df['TotalCharges'].fillna(median_tc, inplace=True)
null_after  = df['TotalCharges'].isnull().sum()

print(f"TotalCharges nulls → before: {null_before}, after: {null_after}")


TotalCharges nulls → before: 11, after: 0


/tmp/ipykernel_196360/3697776791.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalCharges'].fillna(median_tc, inplace=True)


## **Univariate Statistics & Distributions**
### **Numerical Features**

In [6]:
num_cols = ['tenure','MonthlyCharges','TotalCharges']
display(df[num_cols].describe().T.rename(columns={'50%':'median'}))


,count,mean,std,min,25%,median,75%,max
tenure,7043.0,32.371149,24.559481,0.00,9.000,29.000,55.00,72.00
MonthlyCharges,7043.0,64.761692,30.090047,18.25,35.500,70.350,89.85,118.75
TotalCharges,7043.0,2281.916928,2265.270398,18.80,402.225,1397.475,3786.60,8684.80


In [7]:
for col in num_cols:
    fig = px.histogram(
        df, x=col, nbins=50, marginal='box',
        title=f"{col.title()} Distribution & Outliers"
    )
    fig.show()


Tenure is highly right-skewed (median = 29 months), indicating that a large share of customers are relatively new.

MonthlyCharges exhibits two clear peaks, pointing to distinct pricing tiers or plan structures.

TotalCharges shows a pronounced long tail, revealing a small group of high-value customers with very large lifetime spend.

### **Categorical Features**

In [8]:
cat_cols = [c for c in df.columns if df[c].dtype=='object' and c!='customerID']
for col in cat_cols:
    counts = df[col].value_counts().reset_index()
    counts.columns = [col, 'count']
    fig = px.bar(counts, y=col, x='count', orientation='h', title=f"Frequency of {col.title()} Levels", labels={'count':'Count', col:col} )
    fig.update_layout(height=350)
    fig.show()


**Contract Type**

Month-to-month agreements dominate at about 55% of customers, while two-year and one-year contracts account for roughly 24% and 21%, respectively.


**Internet Service**

Fiber-optic subscriptions lead with approximately 44% market share, followed by DSL at 34% and no internet service at 22%.

## **Categorical Encoding**
### **Binary Mapping**

In [9]:
binary_cols = ['Partner','Dependents','PhoneService','PaperlessBilling','Churn']
for col in binary_cols:
    df[col] = df[col].map({'Yes':1,'No':0})

display(df[binary_cols].head(3))


,Partner,Dependents,PhoneService,PaperlessBilling,Churn
0,1,0,0,1,0
1,0,0,1,0,0
2,0,0,1,1,1


**One-Hot Encoding with Rare-Level Grouping**

Rare levels (<1%) grouped into “Other” to control dimensionality.

In [10]:
# Group and one-hot encode
multi_cols = [c for c in df.columns if df[c].dtype=='object' and c!='customerID']

def group_rare(series, threshold=0.01):
    freq = series.value_counts(normalize=True)
    rares = freq[freq < threshold].index
    return series.replace(rares, 'Other')

df_enc = df.copy()
for col in multi_cols:
    df_enc[col] = group_rare(df_enc[col], threshold=0.01)

df_enc = pd.get_dummies(df_enc.drop(columns=['customerID']), drop_first=True)
print(f"Dimensions after encoding: {df_enc.shape[0]:,} × {df_enc.shape[1]}")


Dimensions after encoding: 7,043 × 31


## **Derived Features**
A set of new features inspired by domain knowledge and EDA insights.

### **Tenure Buckets**

The tenure buckets effectively capture the non-linear churn risk evident in the early stages of the customer lifecycle.

In [11]:
bins   = [-1,6,12,24,48,72]
labels = ['0–6','6–12','12–24','24–48','48+']
df_enc['TenureBin'] = pd.cut(df_enc['tenure'], bins=bins, labels=labels)

fig = px.bar(
    df_enc['TenureBin'].value_counts().sort_index(),
    orientation='h', title="Customer Count by Tenure Bucket"
)
fig.show()


### **Service-Count**

The total number of “Yes” service subscriptions to quantify bundle complexity, since a more complex bundle can impact customer satisfaction and churn.

In [12]:
service_flags = [c for c in df_enc.columns if '_Yes' in c and c not in ['Churn']]
df_enc['NumServices'] = df_enc[service_flags].sum(axis=1)

fig = px.histogram(
    df_enc, x='NumServices', nbins=9, marginal='box',
    title="Number of Add-on Services per Customer"
)
fig.show()


### **Revenue Efficiency Metrics**
High per-service revenue could signal up-sell success or price sensitivity.

In [13]:
df_enc['AvgChargePerMonth'] = df_enc['TotalCharges'] / (df_enc['tenure'] + 1)
df_enc['RevPerService'] = df_enc['TotalCharges'] / df_enc['NumServices'].replace(0,1)

fig = px.violin(
    df_enc, x='Churn', y='RevPerService', box=True, points='all',
    title="Revenue per Service by Churn Status"
)
fig.show()


## **Interaction Features**
Several interaction terms to capture combined effects.

### **Contract × InternetService**
Fiber customers on month-to-month plans may churn faster than those locked into multi-year contracts.

In [14]:
df_enc['InternetService_DSL'] = 1 - (
    df_enc['InternetService_Fiber optic'] +
    df_enc['InternetService_No']
)

df_enc['Contract_M2M'] = 1 - (
    df_enc['Contract_One year'] +
    df_enc['Contract_Two year']
)

df_enc['ContractType'] = np.select(
    [
        df_enc['Contract_M2M'] == 1,
        df_enc['Contract_One year'] == 1,
        df_enc['Contract_Two year'] == 1
    ],
    ['Month-to-month', 'One year', 'Two year'],
    default='Unknown'
)

df_enc['InternetType'] = np.select(
    [
        df_enc['InternetService_DSL'] == 1,
        df_enc['InternetService_Fiber optic'] == 1,
        df_enc['InternetService_No'] == 1
    ],
    ['DSL', 'Fiber optic', 'No'],
    default='Other'
)

rates = (
    df_enc
      .groupby(['ContractType', 'InternetType'])['Churn']
      .mean()
      .mul(100)
      .round(1)
      .reset_index()
)

fig = px.bar(
    rates,
    x='InternetType',
    y='Churn',
    color='InternetType',
    facet_col='ContractType',
    category_orders={
        'ContractType': ['Month-to-month', 'One year', 'Two year'],
        'InternetType': ['DSL', 'Fiber optic', 'No']
    },
    labels={
        'Churn': 'Churn Rate (%)',
        'InternetType': 'Internet Service',
        'ContractType': 'Contract Type'
    },
    title='Churn Rate by Internet Service within Each Contract Type',
    text='Churn'
)

fig.update_traces(textposition='outside')
fig.update_xaxes(title_text='Internet Service')
fig.update_layout(
    showlegend=False,
    uniformtext_minsize=8,
    uniformtext_mode='hide'
)

fig.show()


## **Senior Citizen × Long Tenure**

Senior customers with long tenures may exhibit stronger loyalty but may react differently if churned.

In [15]:
df_enc['Senior_LongTerm'] = df_enc['SeniorCitizen'] * (df_enc['tenure'] >= 24).astype(int)
pct = df_enc.groupby('Senior_LongTerm')['Churn'].mean().mul(100).reset_index(name='ChurnPct')
fig = px.bar(pct, x='Senior_LongTerm', y='ChurnPct', title="Churn % for Senior Citizens ≥24mo Tenure")
fig.show()


## **Automatic Billing & Contract**

Convenience of auto-pay may reduce churn, especially on flexible contracts.


In [16]:
df_enc['PaymentMethod_Bank transfer (automatic)'] = 1 - (
    df_enc['PaymentMethod_Credit card (automatic)']
    + df_enc['PaymentMethod_Electronic check']
    + df_enc['PaymentMethod_Mailed check']
)

df_enc['Payment_Auto'] = (
    df_enc['PaymentMethod_Credit card (automatic)']
    + df_enc['PaymentMethod_Bank transfer (automatic)']
).clip(0,1)

df_enc['AutoPay_M2M'] = df_enc['Payment_Auto'] * df_enc['Contract_M2M']

pct = (
    df_enc
      .groupby('AutoPay_M2M')['Churn']
      .mean()
      .mul(100)
      .round(1)
      .reset_index(name='ChurnPct')
)

fig = px.bar(
    pct,
    x='AutoPay_M2M',
    y='ChurnPct',
    title='Churn Rate (%) for Auto-Pay & Month-to-Month Customers',
    labels={'AutoPay_M2M':'Auto-Pay & M2M','ChurnPct':'Churn Rate (%)'},
    text='ChurnPct'
)
fig.update_traces(textposition='outside')
fig.update_layout(yaxis=dict(range=[0, pct['ChurnPct'].max() + 5]))
fig.show()


### **High-Value Customer Flags**

Losing high-value customers has greater revenue impact; patterns may differ.

In [17]:
mc_q75 = df_enc['MonthlyCharges'].quantile(0.75)
tc_q75 = df_enc['TotalCharges'].quantile(0.75)

df_enc['HighMonthlyCharge'] = (df_enc['MonthlyCharges'] >= mc_q75).astype(int)
df_enc['HighTotalCharge']   = (df_enc['TotalCharges']   >= tc_q75).astype(int)
df_enc['HighValueCustomer'] = df_enc['HighMonthlyCharge'] * df_enc['HighTotalCharge']

pct = df_enc.groupby('HighValueCustomer')['Churn'].mean().mul(100).reset_index(name='ChurnPct')
fig = px.bar(pct, x='HighValueCustomer', y='ChurnPct', title="Churn % for High-Value Customers")
fig.show()


## **Partner & Dependents Interaction**

Households with families may value continuity differently.

In [18]:
df_enc['Partner_Dependents'] = df_enc['Partner'] * df_enc['Dependents']
pct = df_enc.groupby('Partner_Dependents')['Churn'].mean().mul(100).reset_index(name='ChurnPct')
fig = px.bar(pct, x='Partner_Dependents', y='ChurnPct', title="Churn % by Partner & Dependents")
fig.show()


## **Feature-Quality Control**
### **Variance Threshold**

In [19]:
numeric_feats = df_enc.select_dtypes(include=[np.number]).columns.tolist()
numeric_feats.remove('Churn')

selector = VarianceThreshold(threshold=0.01)
selector.fit(df_enc[numeric_feats])

kept_feats = [
    feat for feat, keep in zip(numeric_feats, selector.get_support())
    if keep
]

df_qc = df_enc[kept_feats + ['Churn']].copy()
df_qc.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 22 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   SeniorCitizen                            7043 non-null   int64  
 1   Partner                                  7043 non-null   int64  
 2   Dependents                               7043 non-null   int64  
 3   tenure                                   7043 non-null   int64  
 4   PhoneService                             7043 non-null   int64  
 5   PaperlessBilling                         7043 non-null   int64  
 6   MonthlyCharges                           7043 non-null   float64
 7   TotalCharges                             7043 non-null   float64
 8   NumServices                              7043 non-null   int64  
 9   AvgChargePerMonth                        7043 non-null   float64
 10  RevPerService                            7043 no

### **Skewness & Log-Transform**

In [20]:
numeric_feats = df_qc.select_dtypes(include='number').drop(columns=['Churn']).columns
skewed = df_qc[numeric_feats].skew().abs()
high_skew = skewed[skewed > 1].index.tolist()

for feat in high_skew:
    df_qc[f"{feat}_log"] = np.log1p(df_qc[feat])

print("Log-transformed:", high_skew)


Log-transformed: ['SeniorCitizen', 'PhoneService', 'AvgChargePerMonth', 'RevPerService', 'Senior_LongTerm', 'PaymentMethod_Bank transfer (automatic)', 'AutoPay_M2M', 'HighMonthlyCharge', 'HighTotalCharge', 'HighValueCustomer', 'Partner_Dependents']


## **Scaling & Normalization**

In [21]:
num_feats = [c for c in df_qc.columns if c != 'Churn' and df_qc[c].dtype!='object']
scaler_std = StandardScaler().fit(df_qc[num_feats])
scaler_mm  = MinMaxScaler().fit(df_qc[num_feats])

df_scaled = df_qc.copy()
df_scaled[[f"{f}_std" for f in num_feats]] = scaler_std.transform(df_qc[num_feats])
df_scaled[[f"{f}_mm"  for f in num_feats]] = scaler_mm.transform(df_qc[num_feats])

print("Scaled features added (_std and _mm suffixes).")


Scaled features added (_std and _mm suffixes).


## **Multicollinearity Check (VIF)**

In [22]:
X = df_qc.drop(columns=['Churn'])
X_const = sm.add_constant(X)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [
        variance_inflation_factor(X_const.values, i+1)
        for i in range(len(X.columns))
    ]
})

vif_df = vif_df.sort_values('VIF', ascending=False).head(20)
print("Top 20 VIF scores:")
display(vif_df)


/home/kali/anaconda3/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning:

divide by zero encountered in scalar divide



Top 20 VIF scores:


,feature,VIF
0,SeniorCitizen,inf
14,PaymentMethod_Bank transfer (automatic),inf
30,HighValueCustomer_log,inf
29,HighTotalCharge_log,inf
28,HighMonthlyCharge_log,inf
27,AutoPay_M2M_log,inf
26,PaymentMethod_Bank transfer (automatic)_log,inf
25,Senior_LongTerm_log,inf
22,PhoneService_log,inf
21,SeniorCitizen_log,inf


## **Dimensionality Reduction (PCA)**

Partial separation suggests these features capture significant churn variance.

In [23]:
top5_feats = ['tenure_std','MonthlyCharges_std','NumServices_std','AvgChargePerMonth_std','RevPerService_std']
pca = PCA(n_components=2, random_state=42).fit(df_scaled[top5_feats])
coords = pca.transform(df_scaled[top5_feats])
df_pca = pd.DataFrame(coords, columns=['PC1','PC2'])
df_pca['Churn'] = df_scaled['Churn']

fig = px.scatter(df_pca, x='PC1', y='PC2', color='Churn', title="PCA Projection of Top 5 Standardized Features")
fig.show()


## **Preliminary Model-Based Feature Importance**

In [24]:

X_full = df_scaled.drop(columns=['Churn'])
y_full = df_scaled['Churn']

rf = RandomForestClassifier(n_estimators=200, max_depth=7, random_state=42)
rf.fit(X_full, y_full)

importances = pd.Series(rf.feature_importances_, index=X_full.columns).sort_values(ascending=True).head(25)

fig = px.bar(x=importances.values, y=importances.index, orientation='h', title="Top 25 Feature Importances (Random Forest)")
fig.show()


The Random Forest importances reveal a few patterns:

Auto-payment via bank transfer is the single strongest predictor. In fact, every transformation of the PaymentMethod_Bank transfer (automatic) feature, standardized (_std), log-scaled (_log), and min-max scaled (_mm)—occupies the top slots in the importance ranking. This clearly shows that customers who pay automatically by bank transfer exhibit a very different churn behavior compared to other payment methods.

Household composition matters. The interaction Partner_Dependents (i.e. customers with both a partner and dependents) and its log and scaled variants rank immediately behind the payment feature. This suggests that family obligations are an important retention signal—perhaps larger households are more sensitive to service continuity.

Senior long-tenure segment is a key cohort. Features like Senior_LongTerm (senior citizens with tenure ≥ 24 months), again in raw, log, and scaled forms, all appear in the top 10–15. This reinforces the intuition that older, longer-tenured customers have distinct loyalty or churn patterns.

Revenue segmentation plays a role but is secondary. The log-transformed HighValueCustomer shows up mid-chart, indicating that being a high-revenue customer adds information beyond demographics and payment behavior—but it is not as dominant as the auto-pay or household features.

Multiple transformations carry unique signals. The fact that raw, log-scaled, min-max, and z-score versions of the same features all rank highly tells us that not only the level but also the distributional context (e.g. capturing skew or variance) provides incremental splits for the Random Forest.

## **Export Final Preprocessed Data**

In [25]:
out_dir = '../data/processed'
os.makedirs(out_dir, exist_ok=True)
out_fp = os.path.join(out_dir, 'telco_features_final.csv')
df_scaled.to_csv(out_fp, index=False)
print(f"Final feature matrix saved to {out_fp}")


Final feature matrix saved to ../data/processed/telco_features_final.csv
